<a href="https://colab.research.google.com/github/smf-9000/kaggle/blob/main/%5B002%5D_tabular_playground_series_jul_2021_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imread
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from statsmodels.nonparametric.smoothers_lowess import lowess


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jul-2021/train.csv
/kaggle/input/tabular-playground-series-jul-2021/test.csv


In [ ]:
submission_ex = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')

targets_df = train_df[["target_carbon_monoxide","target_benzene","target_nitrogen_oxides"]].copy()
# train_df.drop(["target_carbon_monoxide","target_benzene","target_nitrogen_oxides"], axis=1, inplace=True)  

In [ ]:
def root_mean_squared_log_error(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [ ]:
train_df.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [ ]:
test_df.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1
1,2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0
2,2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8
3,2011-01-01 03:00:00,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0
4,2011-01-01 04:00:00,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3


In [ ]:
targets_df.tail()

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
7106,1.3,5.1,191.1
7107,1.4,5.8,221.3
7108,1.6,5.2,227.4
7109,1.5,4.6,199.8
7110,1.4,4.1,186.5


In [ ]:
submission_ex.head()

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,2.0,10.0,300.0
1,2011-01-01 01:00:00,2.0,10.0,300.0
2,2011-01-01 02:00:00,2.0,10.0,300.0
3,2011-01-01 03:00:00,2.0,10.0,300.0
4,2011-01-01 04:00:00,2.0,10.0,300.0


In [ ]:
train_df.describe()

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
count,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000
mean,20.878034,47.561004,1.110309,1091.572100,938.064970,883.903305,1513.238349,998.335565,2.086219,10.237083,204.066784
std,7.937917,17.398731,0.398950,218.537554,281.978988,310.456355,350.180310,381.537695,1.447109,7.694426,193.927723
min,1.300000,8.900000,0.198800,620.300000,364.000000,310.600000,552.900000,242.700000,0.100000,0.100000,1.900000
25%,14.900000,33.700000,0.855900,930.250000,734.900000,681.050000,1320.350000,722.850000,1.000000,4.500000,76.450000
50%,20.700000,47.300000,1.083500,1060.500000,914.200000,827.800000,1513.100000,928.700000,1.700000,8.500000,141.000000
75%,25.800000,60.800000,1.404150,1215.800000,1124.100000,1008.850000,1720.400000,1224.700000,2.800000,14.200000,260.000000
max,46.100000,90.800000,2.231000,2088.300000,2302.600000,2567.400000,2913.800000,2594.600000,12.500000,63.700000,1472.300000


In [ ]:
test_df.describe()

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
count,2247.000000,2247.000000,2247.000000,2247.000000,2247.000000,2247.000000,2247.000000,2247.000000
mean,10.808144,51.031242,0.627053,1106.534490,836.459769,828.321495,1104.850601,1029.851535
std,6.444497,16.665047,0.266588,205.341455,272.816585,339.511779,293.112225,434.863287
min,-1.800000,9.800000,0.184700,665.900000,356.200000,320.100000,523.400000,218.800000
25%,5.600000,36.900000,0.413350,951.500000,640.700000,597.050000,899.450000,688.550000
50%,9.800000,50.600000,0.596400,1080.400000,800.800000,757.100000,1076.200000,973.100000
75%,14.200000,63.550000,0.804950,1222.100000,1016.100000,944.950000,1288.350000,1324.000000
max,30.900000,88.800000,1.393000,1882.900000,1776.100000,1975.000000,2211.400000,2593.800000


In [ ]:
train_df.isna().sum()

date_time                 0
deg_C                     0
relative_humidity         0
absolute_humidity         0
sensor_1                  0
sensor_2                  0
sensor_3                  0
sensor_4                  0
sensor_5                  0
target_carbon_monoxide    0
target_benzene            0
target_nitrogen_oxides    0
dtype: int64

In [ ]:
test_df.isna().sum()

date_time            0
deg_C                0
relative_humidity    0
absolute_humidity    0
sensor_1             0
sensor_2             0
sensor_3             0
sensor_4             0
sensor_5             0
dtype: int64

### train dataset

In [ ]:
# ################################################################
# for c in train_df.columns:
#     if c != 'date_time':
#         fig = plt.figure(figsize = (15, 5))
#         g = train_df[c].plot()
#         train_df['hmm'] = lowess(train_df[c].values, np.arange(len(train_df[c].values)), frac=0.10)[:, 1]
#         g = train_df['hmm'].plot()
#         g.legend([c])
# del train_df['hmm'] 

In [ ]:
# ################################################################
# fig = plt.figure(figsize = (30,30))
# ax = fig.gca()
# hist = train_df[train_df.columns.values].hist(bins=150, ax = ax)

### test dataset

In [ ]:
# ################################################################
# for c in test_df.columns:
#     if c != 'date_time':
#         fig = plt.figure(figsize = (15, 5))
#         g = test_df[c].plot()
#         test_df['hmm'] = lowess(test_df[c].values, np.arange(len(test_df[c].values)), frac=0.15)[:, 1]
#         g = test_df['hmm'].plot()
#         g.legend([c])
# del test_df['hmm']

In [ ]:
# ################################################################
# fig = plt.figure(figsize = (30,30))
# ax = fig.gca()
# hist = test_df[test_df.columns.values].hist(bins=150, ax = ax)

### feature correlation

In [ ]:
# ################################################################
# corr = train_df.corr()
# fig = plt.figure(figsize = (15,15))
# sns.heatmap(corr, vmax = .8, square = True)
# plt.show()

### new features

In [ ]:
fold_part = 1000
a = train_df.iloc[1:fold_part,:].iloc[::-1]
b = train_df.iloc[-fold_part:-1,:].iloc[::-1]
train_df = a.append(train_df.append(b))
a = test_df.iloc[1:fold_part,:].iloc[::-1]
b = test_df.iloc[-fold_part:-1,:].iloc[::-1]
test_df = a.append(test_df.append(b))
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# -------------------------------------------------

for df in [train_df, test_df]:
    for col in df.columns:
        if col not in ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides','date_time']:
            for mean in [0.03, 0.1, 0.2]:
                df['mean_' + str(mean) + '_' + col] = lowess(df[col].values, np.arange(len(df[col].values)), frac=mean)[:, 1]
            
    df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')
    
    df['deg_F'] = (df['deg_C']*(9/5)) + 32
    df["SMC"] = (df["absolute_humidity"] * 100) / df['relative_humidity']
    df['dew_point'] = df['deg_C'] - ((100 - df['relative_humidity']) / 5)
    
    df['time'] = df['date_time'].astype(np.int64)//10**9
    df['hh'] = df['date_time'].dt.hour.values
    df['rush_h'] = df['hh'].apply(lambda x: 1 if x in [8,9,18,19,20]  else 0)
    df['day_of_w'] = df['date_time'].dt.dayofweek.values
    df['weekend'] = df['day_of_w'].apply(lambda x: 1 if (x>4)  else 0)
    df['day_of_m'] = df['date_time'].dt.day.values
    df['month'] = df['date_time'].dt.month.values
    
    df.drop(['dew_point'], axis=1, inplace=True)  
    
    df['hm_1'] = ((df['absolute_humidity'].values < 0.25)).astype(int)
    
#     for sen in ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']:
#         df.loc[df['hm_1'] == 1, sen] = df['mean_0.1_' + sen]
    
    lag_features = ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
    lags = [3, 6, 9, 12]
    for feature in lag_features:
        for lag in lags:
            df[feature + '_lag_' + str(lag)] = df[feature].shift(periods=lag, fill_value = 0)

# -------------------------------------------------

train_df = train_df.iloc[fold_part - 1:-fold_part + 1,:]
test_df = test_df.iloc[fold_part - 1:-fold_part + 1,:]
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
# from sklearn import preprocessing
# # scaler = preprocessing.MinMaxScaler()
# # scaler = preprocessing.MaxAbsScaler()
# scaler = preprocessing.StandardScaler()
# for df in [train_df, test_df]:
#     for c in df.columns:
#         if c not in ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides','date_time']:
#             df[[c]] = scaler.fit_transform(df[[c]])

In [ ]:
# train_df.head()

In [ ]:
# test_df.head()

In [ ]:
# lgbm = LGBMRegressor()
# _target = 'target_carbon_monoxide'
# X = np.array(train_df[list(test_df.columns.values)[1:]].values, dtype='float32')
# y = np.squeeze(np.array(train_df[[_target]].values, dtype='float32'))
# params = {
#     'num_leaves': [16, 21, 26],
#     'learning_rate': [0.001, 0.003, 0.005],
#     'max_depth': [6, 7, 8],
#     'n_estimators': [700, 1000, 1300],
#     'min_data_in_leaf': [5, 20]
# }
# grid= GridSearchCV(estimator=lgbm, param_grid = params, cv = 5, n_jobs=-1, verbose=True)
# grid.fit(X, y)
# print('=====================================', 'target: ', _target)
# print(grid.best_estimator_)
# print('-------------------------------------')
# print(grid.best_score_)
# print('-------------------------------------')
# print(grid.best_params_)

In [ ]:
# lgbm = LGBMRegressor()
# _target = 'target_benzene'
# X = np.array(train_df[list(test_df.columns.values)[1:]].values, dtype='float32')
# y = np.squeeze(np.array(train_df[[_target]].values, dtype='float32'))
# params = {
#     'num_leaves': [3, 5, 7],
#     'learning_rate': [0.003, 0.01, 0.03],
#     'max_depth': [3, 4, 5],
#     'n_estimators': [700, 1000, 1300]
# }
# grid= GridSearchCV(estimator=lgbm, param_grid = params, cv = 5, n_jobs=-1, verbose=True)
# grid.fit(X, y)
# print('=====================================', 'target: ', _target)
# print(grid.best_estimator_)
# print('-------------------------------------')
# print(grid.best_score_)
# print('-------------------------------------')
# print(grid.best_params_)

In [ ]:
# lgbm = LGBMRegressor()
# _target = 'target_nitrogen_oxides'
# X = np.array(train_df[list(test_df.columns.values)[1:]].values, dtype='float32')
# y = np.squeeze(np.array(train_df[[_target]].values, dtype='float32'))
# params = {
#     'num_leaves': [3, 4, 5],
#     'learning_rate': [0.2, 0.1, 0.05],
#     'max_depth': [-1, 5, 10],
#     'n_estimators': [300, 500, 800]
# }
# grid= GridSearchCV(estimator=lgbm, param_grid = params, cv = 5, n_jobs=-1, verbose=True)
# grid.fit(X, y)
# print('=====================================', 'target: ', _target)
# print(grid.best_estimator_)
# print('-------------------------------------')
# print(grid.best_score_)
# print('-------------------------------------')
# print(grid.best_params_)

### LGBMRegressor

In [ ]:
# ===================================== target:  target_carbon_monoxide
# LGBMRegressor(learning_rate=0.005, max_depth=7, min_data_in_leaf=20,
#               n_estimators=1300, num_leaves=26)
# -------------------------------------
# 0.8820984830544785
# -------------------------------------
# {'learning_rate': 0.005, 'max_depth': 7, 'min_data_in_leaf': 20, 'n_estimators': 1300, 'num_leaves': 26}

# ===================================== target:  target_benzene
# LGBMRegressor(learning_rate=0.03, max_depth=3, n_estimators=700, num_leaves=7)
# -------------------------------------
# 0.9719938458962158
# -------------------------------------
# {'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 700, 'num_leaves': 7}
# ===================================== target:  target_nitrogen_oxides
# LGBMRegressor(n_estimators=300, num_leaves=4)
# -------------------------------------
# 0.5840828704905612
# -------------------------------------
# {'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 300, 'num_leaves': 4}

In [ ]:
lgbm_regs = {}

lgbm_regs['target_carbon_monoxide'] = LGBMRegressor(
                        num_leaves=26,
                        learning_rate=0.005,
                        max_depth= 7,
                        n_estimators=1300)

lgbm_regs['target_benzene'] = LGBMRegressor(
                        num_leaves=7,
                        learning_rate=0.03,
                        max_depth= 3,
                        n_estimators=700)

lgbm_regs['target_nitrogen_oxides'] = LGBMRegressor(
                        num_leaves=4,
                        learning_rate=0.1,
                        max_depth= -1,
                        n_estimators=300)


### submission

In [ ]:
submit = True

In [ ]:
submission = submission_ex.copy()

for _target in ['target_carbon_monoxide', 'target_benzene','target_nitrogen_oxides']:
    X = np.array(train_df[list(test_df.columns.values)[1:]].values, dtype='float32')
    y = np.squeeze(np.array(targets_df[[_target]].values, dtype='float32'))
    X_test = np.array(test_df.iloc[:,1:].values, dtype='float32')
    if not submit:
#         X, X_val, y, y_val = train_test_split(X, y, test_size=0.2, random_state=423)
        X, X_val = X[:5500], X[5500:]
        y, y_val = y[:5500], y[5500:]
        lgbm_regs[_target].fit(X, y)
        p1 = lgbm_regs[_target].predict(X_val)
        print('target: ', _target)
        print('rmsle: ', root_mean_squared_log_error(y_val, p1))
        
        fig = plt.figure(figsize = (30, 15))
        g = targets_df.iloc[5500:,:].reset_index()[_target].plot()
        tmp_col = pd.DataFrame(data=p1, columns=['tmp'])
        g = tmp_col['tmp'].plot(alpha = 0.7)
        
        g.legend([_target])
    else:
        print('submit: ', _target)
        lgbm_regs[_target].fit(X, y)
        p1 = lgbm_regs[_target].predict(X_test)
        submission[_target] = p1


submit:  target_carbon_monoxide
submit:  target_benzene
submit:  target_nitrogen_oxides


In [ ]:
submission.to_csv('submission.csv', index=False)

test = pd.read_csv('submission.csv')
test.tail()

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
2242,2011-04-04 10:00:00,2.930582,13.008469,590.232949
2243,2011-04-04 11:00:00,2.545479,10.546841,412.216426
2244,2011-04-04 12:00:00,2.887104,12.697983,409.606681
2245,2011-04-04 13:00:00,2.252789,10.531495,375.900760
2246,2011-04-04 14:00:00,2.768202,13.023273,375.989197
